In [1]:
import pandas as pd
data = pd.read_excel("C:\\Users\\mukes\\Downloads\\Association Rules\\Association Rules\\Online retail.xlsx")
data

,"shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil"
0,"burgers,meatballs,eggs"
1,chutney
2,"turkey,avocado"
3,"mineral water,milk,energy bar,whole wheat rice..."
4,low fat yogurt
...,...
7495,"butter,light mayo,fresh bread"
7496,"burgers,frozen vegetables,eggs,french fries,ma..."
7497,chicken
7498,"escalope,green tea"


In [2]:
print(data.head())

  shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
0                             burgers,meatballs,eggs                                                                                                                                                                             
1                                            chutney                                                                                                                                                                             
2                                     turkey,avocado                                                                                                                                                                             
3  mineral water,milk,energy bar,whole wheat rice...                                            

In [3]:
# Check for missing values
print(data.isnull().sum())


shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil    0
dtype: int64


In [4]:
# Check the columns in the dataset
print(data.columns)

# Remove rows with missing values in key columns if they exist
required_columns = ['InvoiceNo', 'StockCode', 'Quantity']
missing_columns = [col for col in required_columns if col not in data.columns]

if not missing_columns:
    data.dropna(subset=required_columns, inplace=True)
else:
    print(f"Missing columns: {missing_columns}")

# Remove duplicate rows
data.drop_duplicates(inplace=True)

# Check if 'InvoiceNo' exists before filtering cancelled orders
if 'InvoiceNo' in data.columns:
    data = data[~data['InvoiceNo'].str.startswith('C')]
else:
    print("'InvoiceNo' column not found, skipping cancellation filter.")


Index(['shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil'], dtype='object')
Missing columns: ['InvoiceNo', 'StockCode', 'Quantity']
'InvoiceNo' column not found, skipping cancellation filter.


In [5]:
# Check the columns in the DataFrame after preprocessing
print(data.columns)

# Ensure 'InvoiceNo' and 'StockCode' exist before creating the basket
if 'InvoiceNo' in data.columns and 'StockCode' in data.columns:
    # Create a transaction list
    basket = data.groupby(['InvoiceNo', 'StockCode'])['Quantity'].sum().unstack().reset_index().fillna(0)

    # Convert quantities to 1s and 0s (1 if item was purchased, 0 otherwise)
    basket = basket.set_index('InvoiceNo').applymap(lambda x: 1 if x > 0 else 0)
else:
    missing_columns = [col for col in ['InvoiceNo', 'StockCode'] if col not in data.columns]
    print(f"Missing columns: {missing_columns}. Cannot create transaction list.")


Index(['shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil'], dtype='object')
Missing columns: ['InvoiceNo', 'StockCode']. Cannot create transaction list.


In [6]:
!pip install mlxtend


Defaulting to user installation because normal site-packages is not writeable


In [7]:
# Import necessary libraries
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

# Ensure the basket DataFrame is created properly
# Assuming the basket variable is already created from previous steps

# Check if the basket DataFrame exists
if 'basket' in locals():
    # Find frequent itemsets with a minimum support of 0.01
    frequent_itemsets = apriori(basket, min_support=0.01, use_colnames=True)

    # Generate association rules with a minimum confidence of 0.5
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

    # Display the first few rules
    print(rules.head())
else:
    print("Basket DataFrame not found. Please ensure it was created correctly.")


Basket DataFrame not found. Please ensure it was created correctly.


In [9]:
# Import necessary libraries
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

# Load the dataset
data = pd.read_excel("C:\\Users\\mukes\\Downloads\\Association Rules\\Association Rules\\Online retail.xlsx")

# Data Preprocessing
# Check the columns in the dataset
print("Columns in dataset:", data.columns)

# Remove rows with missing values in key columns if they exist
required_columns = ['InvoiceNo', 'StockCode', 'Quantity']
missing_columns = [col for col in required_columns if col not in data.columns]

if not missing_columns:
    data.dropna(subset=required_columns, inplace=True)
else:
    print(f"Missing columns: {missing_columns}")

# Remove duplicate rows
data.drop_duplicates(inplace=True)

# Filter out cancelled orders
if 'InvoiceNo' in data.columns:
    data = data[~data['InvoiceNo'].str.startswith('C')]
else:
    print("'InvoiceNo' column not found, skipping cancellation filter.")

# Create a transaction list
if 'InvoiceNo' in data.columns and 'StockCode' in data.columns:
    # Create a basket DataFrame
    basket = data.groupby(['InvoiceNo', 'StockCode'])['Quantity'].sum().unstack().reset_index().fillna(0)
    # Convert quantities to 1s and 0s (1 if item was purchased, 0 otherwise)
    basket = basket.set_index('InvoiceNo').applymap(lambda x: 1 if x > 0 else 0)
    print("Basket DataFrame created successfully.")
else:
    print("Missing columns. Cannot create transaction list.")

# Apply Apriori Algorithm
if 'basket' in locals():
    # Find frequent itemsets with a minimum support of 0.01
    frequent_itemsets = apriori(basket, min_support=0.01, use_colnames=True)

    # Generate association rules with a minimum confidence of 0.5
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

    # Display the first few rules
    print(rules.head())
else:
    print("Basket DataFrame not found. Please ensure it was created correctly.")


C:\ProgramData\anaconda3\Lib\site-packages\openpyxl\packaging\core.py:99: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow()
C:\ProgramData\anaconda3\Lib\site-packages\openpyxl\packaging\core.py:99: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow()


Columns in dataset: Index(['shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil'], dtype='object')
Missing columns: ['InvoiceNo', 'StockCode', 'Quantity']
'InvoiceNo' column not found, skipping cancellation filter.
Missing columns. Cannot create transaction list.
Basket DataFrame not found. Please ensure it was created correctly.


In [13]:
# Import necessary libraries
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

# Data Preprocessing
print("Columns in dataset:", data.columns)

required_columns = ['InvoiceNo', 'StockCode', 'Quantity']
missing_columns = [col for col in required_columns if col not in data.columns]

if not missing_columns:
    data.dropna(subset=required_columns, inplace=True)
else:
    print(f"Missing columns: {missing_columns}")

data.drop_duplicates(inplace=True)

if 'InvoiceNo' in data.columns:
    data = data[~data['InvoiceNo'].str.startswith('C')]
else:
    print("'InvoiceNo' column not found, skipping cancellation filter.")

# Create a transaction list
if 'InvoiceNo' in data.columns and 'StockCode' in data.columns:
    basket = data.groupby(['InvoiceNo', 'StockCode'])['Quantity'].sum().unstack().reset_index().fillna(0)
    basket = basket.set_index('InvoiceNo').applymap(lambda x: 1 if x > 0 else 0)
    print("Basket DataFrame created successfully.")
else:
    print("Missing columns. Cannot create transaction list.")

# Apply Apriori Algorithm
if 'basket' in locals():
    frequent_itemsets = apriori(basket, min_support=0.01, use_colnames=True)

    # Generate association rules with a minimum confidence of 0.5
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

    if not rules.empty:
        # Display the first few rules
        print("First few rules:")
        print(rules.head())

        # Filtering rules for better insights
        rules_filtered = rules[(rules['support'] >= 0.01) & (rules['confidence'] >= 0.5) & (rules['lift'] >= 1)]
        print("Filtered rules:")
        print(rules_filtered)
    else:
        print("No rules generated. Please check the frequent itemsets.")
else:
    print("Basket DataFrame not found. Please ensure it was created correctly.")


Columns in dataset: Index(['shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil'], dtype='object')
Missing columns: ['InvoiceNo', 'StockCode', 'Quantity']
'InvoiceNo' column not found, skipping cancellation filter.
Missing columns. Cannot create transaction list.
Basket DataFrame not found. Please ensure it was created correctly.


In [15]:
# Import necessary libraries
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

# Data Preprocessing
print("Columns in dataset:", data.columns)

# Define required columns
required_columns = ['InvoiceNo', 'StockCode', 'Quantity']
missing_columns = [col for col in required_columns if col not in data.columns]

if not missing_columns:
    data.dropna(subset=required_columns, inplace=True)
else:
    print(f"Missing columns: {missing_columns}")

# Remove duplicate rows
data.drop_duplicates(inplace=True)

# Filter out cancelled orders
if 'InvoiceNo' in data.columns:
    data = data[~data['InvoiceNo'].str.startswith('C')]
else:
    print("'InvoiceNo' column not found, skipping cancellation filter.")

# Create a transaction list
if 'InvoiceNo' in data.columns and 'StockCode' in data.columns:
    basket = data.groupby(['InvoiceNo', 'StockCode'])['Quantity'].sum().unstack().reset_index().fillna(0)
    basket = basket.set_index('InvoiceNo').applymap(lambda x: 1 if x > 0 else 0)
    print("Basket DataFrame created successfully.")
else:
    print("Missing columns. Cannot create transaction list.")

# Apply Apriori Algorithm
if 'basket' in locals():
    frequent_itemsets = apriori(basket, min_support=0.01, use_colnames=True)
    print("Frequent itemsets generated.")

    # Generate association rules with a minimum confidence of 0.5
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

    if not rules.empty:
        # Display the first few rules
        print("First few rules:")
        print(rules.head())

        # Filtering rules for better insights
        rules_filtered = rules[(rules['support'] >= 0.01) & (rules['confidence'] >= 0.5) & (rules['lift'] >= 1)]
        print("Filtered rules:")
        print(rules_filtered)
    else:
        print("No rules generated. Please check the frequent itemsets.")
else:
    print("Basket DataFrame not found. Please ensure it was created correctly.")


Columns in dataset: Index(['shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil'], dtype='object')
Missing columns: ['InvoiceNo', 'StockCode', 'Quantity']
'InvoiceNo' column not found, skipping cancellation filter.
Missing columns. Cannot create transaction list.
Basket DataFrame not found. Please ensure it was created correctly.


In [21]:


# Data Preprocessing
print("Columns in dataset:", data.columns)

# Define required columns
required_columns = ['InvoiceNo', 'StockCode', 'Quantity']
missing_columns = [col for col in required_columns if col not in data.columns]

if not missing_columns:
    data.dropna(subset=required_columns, inplace=True)
else:
    print(f"Missing columns: {missing_columns}")

# Remove duplicate rows
data.drop_duplicates(inplace=True)

# Filter out cancelled orders
if 'InvoiceNo' in data.columns:
    data = data[~data['InvoiceNo'].str.startswith('C')]
else:
    print("'InvoiceNo' column not found, skipping cancellation filter.")

# Create a transaction list
if 'InvoiceNo' in data.columns and 'StockCode' in data.columns:
    basket = data.groupby(['InvoiceNo', 'StockCode'])['Quantity'].sum().unstack().reset_index().fillna(0)
    basket = basket.set_index('InvoiceNo').applymap(lambda x: 1 if x > 0 else 0)
    print("Basket DataFrame created successfully.")
else:
    print("Missing columns. Cannot create transaction list.")

# Apply Apriori Algorithm
if 'basket' in locals():
    frequent_itemsets = apriori(basket, min_support=0.01, use_colnames=True)
    print("Frequent itemsets generated.")

    # Generate association rules with a minimum confidence of 0.5
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

    if not rules.empty:
        # Display the first few rules
        print("First few rules:")
        print(rules.head())

        # Filtering rules for better insights
        rules_filtered = rules[(rules['support'] >= 0.01) & (rules['confidence'] >= 0.5) & (rules['lift'] >= 1)]
        
        if not rules_filtered.empty:
            print("Filtered rules:")
            print(rules_filtered)

            # Visualization
            top_rules = rules_filtered.nlargest(10, 'lift')

            plt.figure(figsize=(10, 6))
            sns.barplot(x='lift', y='antecedents', data=top_rules, palette='viridis')
            plt.title('Top 10 Association Rules by Lift')
            plt.xlabel('Lift')
            plt.ylabel('Rules (Antecedents)')
            plt.show()
        else:
            print("No filtered rules found. Check your filtering criteria.")
    else:
        print("No rules generated. Please check the frequent itemsets.")
else:
    print("Basket DataFrame not found. Please ensure it was created correctly.")


Columns in dataset: Index(['shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil'], dtype='object')
Missing columns: ['InvoiceNo', 'StockCode', 'Quantity']
'InvoiceNo' column not found, skipping cancellation filter.
Missing columns. Cannot create transaction list.
Basket DataFrame not found. Please ensure it was created correctly.


In [ ]:
INTERVIEW QUESTIONS


 1.What is lift and why is it important in Association rules?

  Ans:Lift is a measure of the strength of an association between two items in data mining and association rule learning.

      It's calculated by dividing the confidence of the association by the expected confidence of the rule.

      Lift is important because it helps determine if the association between two items is stronger than expected, and therefore may be worth further investigation.

      A lift value greater than 1 indicates a strong association, while a value less than 1 indicates a weak association. 


 2.What is support and Confidence. How do you calculate them?
    
  Ans:Support and confidence are metrics used in data mining to identify important relationships in data.
    
      Support measures how often an item appears in a dataset, while confidence measures how often an if-then statement is true. 

      Confidence is calculated by dividing the number of transactions containing both itemsets by the number of transactions containing the first itemset.

      Support is used to identify itemsets that occur frequently in the dataset.

  3.What are some limitations or challenges of Association rules mining?
          
  High Computational Complexity: The process can be computationally intensive, especially for large datasets.

  Scalability Issues: As the size of the dataset increases, the number of possible itemsets grows exponentially.
                                                  
  Parameter Sensitivity: The results of association rule mining are highly sensitive to the choice of support, confidence, and lift thresholds.